# Building a Spam Filter with Naive Bayes

In this project, we're going to build a spam filter for SMS messages using the **Naive Bayes** algorithm. 

To classify messages as spam or non-spam: 
- Learns how humans classify messages
- Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
- Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository]('https://archive.ics.uci.edu/ml/datasets/sms+spam+collection')

## 1- Exploring Dataset

In [101]:
import pandas as pd

sms = pd.read_csv('SMSSpamCollection',sep='\t',header=None,names=['Label','SMS'])
print(sms.shape)
print(sms['Label'].value_counts(normalize=True)*100)
sms.head()

(5572, 2)
ham     86.593683
spam    13.406317
Name: Label, dtype: float64


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


After checking our dataset, we saw that 87% of the messages are ham (means non-spam) and 13% are spam. Now we're familiar with the dataset, we can start building the spam filter.

## 2-Training and Test Set

We're going to split our dataset into a training(80% of the dataset) and a test(20% of the dataset) set.
 

In [102]:
# Randomize the dataset
random_sms = sms.sample(frac=1,random_state=1)

# calculate the index for split
training_test_index = round(len(random_sms) * 0.8)

# Split the different set
training_set = random_sms[:training_test_index].reset_index(drop=True)
test_set = random_sms[training_test_index:].reset_index(drop=True)

print('=====TRAINING======\n')
print(training_set.shape)
print(training_set['Label'].value_counts(normalize=True)*100)
print('=====TEST======\n')
print(test_set.shape)
print(test_set['Label'].value_counts(normalize=True)*100)

=====TRAINING======

(4458, 2)
ham     86.54105
spam    13.45895
Name: Label, dtype: float64
=====TEST======

(1114, 2)
ham     86.804309
spam    13.195691
Name: Label, dtype: float64


In the differents set, we can see the percentages of spam and ham are close to the percentages in the full dataset.

## 3- Letter Case and Punctuation

To use easily our algorithm we should do some data cleaning to bring the data from this format: 

![current format](old_format.png)

to :

![new format](new_format.png)

First, let's remove the punctuation and change all letters to lowercase.

In [103]:
import re
# Remove all the punctuation
training_set['SMS'] = training_set['SMS'].apply(lambda x: re.sub('\W',' ',x))
# Transform every letter in every word to lower case
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## 4- Creating the Vocabulary

Second, Let's create a vocabulary *(the unique words)* for the messages in the training set.


In [104]:
# Transform each message from the SMS column
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for words in training_set['SMS']:
    for word in words:
        vocabulary.append(word)

vocabulary = list(set(vocabulary)) 
print(len(vocabulary))     
print(vocabulary[:20])

7783
['impressed', 'ryder', '07090201529', 'minmoremobsemspobox45po139wa', 'w1j', 'abt', 'txtstop', 'masteriastering', 'usmle', 'transfer', 'endowed', 'cops', 'stream', 'vl', 'mini', 'prepayment', 'messaged', 'fullonsms', 'withdraw', 'otherwise']


In our dictionary, we've **7783** unique words from our training set.

## 5- The Final Training Set


In [105]:
word_counts_per_sms = {unique_word: [0]*len(training_set['SMS']) 
for unique_word in vocabulary}

for index,sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] +=1
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,impressed,ryder,07090201529,minmoremobsemspobox45po139wa,w1j,abt,txtstop,masteriastering,usmle,transfer,...,moral,flavour,09064011000,club4mobiles,gpu,toshiba,difference,unconsciously,doke,hahaha
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
# Concatenate the training set to the vocabulary 
training_set_clean = pd.concat([training_set,word_counts],axis=1)
training_set_clean.head()

,Label,SMS,impressed,ryder,07090201529,minmoremobsemspobox45po139wa,w1j,abt,txtstop,masteriastering,...,moral,flavour,09064011000,club4mobiles,gpu,toshiba,difference,unconsciously,doke,hahaha
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 6- Calculating Constants First

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:
$$
P(Spam | w_1,w, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
$$
$$
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
$$

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we'll need to use these equations:
$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$
$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:
- P(Spam) and P(Ham)
- $N_{Spam}$, $N_{Ham}$, $N_{Vocabulary}$

We'll also use Laplace smoothing and set $\alpha = 1$.

In [107]:
# Isolate the differents messages
spam = training_set_clean[training_set_clean['Label']=='spam']
ham = training_set_clean[training_set_clean['Label']=='ham']
# Probability of spam message
p_spam = (len(spam) / len(training_set_clean) ) * 100
# Probability of ham message
p_ham = (len(ham) / len(training_set_clean) ) * 100
print('====== PROBABILITIES ======')
print(p_spam)
print(p_ham)
print('\n')

number_spam_words= sum(spam['SMS'].apply(len))
number_ham_words =  sum(ham['SMS'].apply(len))
number_vocabulary_words = len(vocabulary)
print(number_spam_words)
print(number_ham_words)
print(number_vocabulary_words)

# Laplace smoothing
alpha = 1

====== PROBABILITIES ======
13.458950201884253
86.54104979811575


15190
57237
7783


## 7- Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:
$$
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$
$$
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$

In [108]:
parameter_spam = {word: 0 for word in vocabulary}
parameter_ham = {word: 0 for word in vocabulary}

for word in vocabulary:
    count_word_spam = spam[word].sum()
    count_word_ham = ham[word].sum()
    parameter_spam[word] = (count_word_spam + alpha) / (number_spam_words + (alpha * number_vocabulary_words)) 
    parameter_ham[word] = (count_word_ham + alpha) / (number_ham_words + (alpha * number_vocabulary_words))
print(type(count_word_spam))
print(type(alpha))

<class 'numpy.int64'>
<class 'int'>


## 8- Classifying A New Message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)

- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)

- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
  
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

In [109]:
def classify(message):
    message = re.sub('/W',' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message :
        if word in vocabulary:
            if word in parameter_ham: p_ham_given_message *= parameter_ham[word]
            if word in parameter_spam: p_spam_given_message *= parameter_spam[word]
            
        
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

# Test the function
classify('WINNER!! This is the secret code to unlock the money: C3421.')
classify("Sounds good, Tom, then see u there")

P(Spam|message): 1.0164097981708961e-16
P(Ham|message): 1.819563818233026e-17
Label: Spam
P(Spam|message): 5.359472501724851e-16
P(Ham|message): 2.8089018273976986e-12
Label: Ham


## 9- Measuring the Spam Filter's Accuracy

We done creating a spam filter, and classify 2 messages. Now we'll try to determine how well the spam filter does on our test set of 1,114 messages.


In [110]:
# Change the returns of the function 
def classify_test_set(message):
    message = re.sub('/W',' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    for word in message :
        if word in vocabulary:
            if word in parameter_ham: p_ham_given_message *= parameter_ham[word]
            if word in parameter_spam: p_spam_given_message *= parameter_spam[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

# clean the test set 
test_set['SMS'] = test_set['SMS'].apply(lambda x: re.sub('\W',' ',x)).str.lower()
# create a new column in our test set
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)

test_set.head()

,Label,SMS,predicted
0,ham,later i guess i needa do mcat study too,ham
1,ham,but i haf enuff space got like 4 mb,ham
2,spam,had your mobile 10 mths update to latest oran...,spam
3,ham,all sounds good fingers makes it difficult ...,ham
4,ham,all done all handed in don t know if mega sh...,ham


In [111]:
# Let's calculate the accuracy of the spam filter
correct = 0 
total = test_set.shape[0]

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']: correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
accuracy = (correct / total) * 100
print('The accuracy of the spam filter : {}'.format(accuracy))

Correct: 1100
Incorrect: 14
The accuracy of the spam filter : 98.74326750448833


Seeing the accuracy that reach **98.743 %**, we can conclude our spam filter works well. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.

## 10- Why incorrect classification

Let's figure out why the algorithm reached the wrong conclusions on the 14 messages.

In [112]:
incorrect = test_set[test_set['predicted'] != test_set['Label']]
false_spam = incorrect[(incorrect['predicted']=='spam')&(incorrect['Label']=='ham')].reset_index(drop=True)
false_ham = incorrect[(incorrect['predicted']=='ham')&(incorrect['Label']=='spam')].reset_index(drop=True)
unclear = incorrect[incorrect['predicted']=='needs human classification'].reset_index(drop=True)

print('_________________________________________________________________________\n')
print('UNCLEAR MESSAGES:')
for row in unclear.iterrows():
    print(f'{row[0]+1}. ', row[1]['SMS'])
print('_________________________________________________________________________\n')
print('FALSE HAM MESSAGES:')
for row in false_ham.iterrows():
    print(f'{row[0]+1}. ', row[1]['SMS'])
print('_________________________________________________________________________\n')
print('FALSE SPAM MESSAGES:')
for row in false_spam.iterrows():
    print(f'{row[0]+1}. ', row[1]['SMS'])
print('_________________________________________________________________________')

_________________________________________________________________________

UNCLEAR MESSAGES:
1.  a boy loved a gal  he propsd bt she didnt mind  he gv lv lttrs  bt her frnds threw thm  again d boy decided 2 aproach d gal   dt time a truck was speeding towards d gal  wn it was about 2 hit d girl d boy ran like hell n saved her  she asked  hw cn u run so fast   d boy replied  boost is d secret of my energy  n instantly d girl shouted  our energy  n thy lived happily 2gthr drinking boost evrydy moral of d story   i hv free msgs d    gud ni8
_________________________________________________________________________

FALSE HAM MESSAGES:
1.  not heard from u4 a while  call me now am here all night with just my knickers on  make me beg for it like u did last time 01223585236 xx luv nikiyu4 net
2.  more people are dogging in your area now  call 09090204448 and join like minded guys  why not arrange 1 yourself  there s 1 this evening  a 1 50 minapn ls278bb
3.  oh my god  i ve found your number a

- Unclear messages: they characterized to be long and have a lot of slang and abbreviation that are not probably in the vocabulary. 
- False Ham messages: they tend to be rather long and have a high percentage of "normal" words, which allows them to override the system.
- False Spam messages: they are very short and contain suspicious ad-style words like calls,unlimited,contact us etc. those words are mostly found in spam messages.

## 11- Exp : Making the Algorithm sensitive to letter case
We're going to update our spam filter by making the algorithm sensitive to letter case and see the result. To do that we are not going to transform the letter to lower case in our training and test set.

In [113]:
# Our new data set
training_set_exp = random_sms[:training_test_index].reset_index(drop=True)
test_set_exp = random_sms[training_test_index:].reset_index(drop=True)

# Only to Remove the punctuation
training_set_exp['SMS'] = training_set_exp['SMS'].apply(lambda x: re.sub('\W',' ',x))
test_set_exp['SMS'] = test_set_exp['SMS'].apply(lambda x: re.sub('\W',' ',x))
# Create the vocabulary
vocabulary_exp = []
for words in training_set_exp['SMS']:
    for word in words:
        vocabulary_exp.append(word)
vocabulary_exp = list(set(vocabulary_exp))

# Count word
word_counts_per_sms_exp = {unique_word: [0] * len(training_set_exp['SMS']) for unique_word in vocabulary_exp}
for index,sms in enumerate(training_set_exp['SMS']):
    for word in sms:
        word_counts_per_sms_exp[word][index] += 1
word_counts_exp = pd.DataFrame(word_counts_per_sms_exp)
# Concatenate the training set to the vocabulary
training_set_final_exp = pd.concat([training_set_exp,word_counts_exp],axis=1)

# Isolate the differents messages
spam_sms_exp = training_set_final_exp[training_set_final_exp['Label']=='spam']
ham_sms_exp = training_set_final_exp[training_set_final_exp['Label']=='ham']
# Probability of spam message
p_spam_exp = (len(spam_sms_exp) / len(training_set_final_exp) ) * 100
# Probability of ham message
p_ham_exp = (len(ham_sms_exp) / len(training_set_final_exp) ) * 100

number_spam_words= sum(spam_sms_exp['SMS'].apply(len))
number_ham_words =  sum(ham_sms_exp['SMS'].apply(len))
number_vocabulary_words = len(vocabulary_exp)

# Calculation of the parameters
parameter_spam = {}
parameter_ham = {}

for word in vocabulary_exp:
    count_word_spam = spam_sms_exp[word].sum()
    count_word_ham = ham_sms_exp[word].sum()
    parameter_spam[word]= (count_word_spam + alpha) / (number_ham_words + (alpha * number_vocabulary_words))
    parameter_ham[word] = (count_word_ham + alpha) / (number_ham_words + (alpha * number_vocabulary_words))
#print(vocabulary)

In [114]:
# Spam filter 
def classify_test_set_exp(message):
    message = re.sub('/W',' ', message)
    message = message.split()
    
    p_spam_given_message = p_spam_exp
    p_ham_given_message = p_ham_exp
    for word in message :
        if word in vocabulary:
            if word in parameter_ham: p_ham_given_message *= parameter_ham[word]
            if word in parameter_spam: p_spam_given_message *= parameter_spam[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

# Create a new column in our test set
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)

# Let's calculate the accuracy of the spam filter
correct = 0 
total = test_set.shape[0]

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']: correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
accuracy = (correct / total) * 100
print('The accuracy of the spam filter : {}'.format(accuracy))

Correct: 963
Incorrect: 151
The accuracy of the spam filter : 86.44524236983841


This experience show us the fact to make the filter sensitive to letter case reduce the accuracy of the filter (the accuracy has dropped by *12.29%*). It seems that the letter case doesn't really make any valuable difference when it comes to distinguishing between spam and ham messages.

# Conclusion
In this project, we created a spam filter based on the multinomial Naive Bayes algorithmwith with an accuracy of **98.743%**, which is almost 20% higher than our goal (accuracy of 80%).